In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai import *
from fastai.vision import *
import torch
from torch import nn
from dsin.ae.data_manager.data_loader import (
    SideinformationImageImageList, ImageSiTuple)
from dsin.ae import config
from dsin.ae.si_ae import SideInformationAutoEncoder
from dsin.ae.si_net import SiNetChannelIn
from dsin.ae.loss_man import LossManager
from dsin.ae.distortions import Distortions
from dsin.ae.kitti_normalizer import ChangeImageStatsToKitti, ChangeState
from dsin.ae import config


In [3]:
config.use_si_flag

<SiNetChannelIn.NoSideInformation: 3>

In [4]:
si_autoencoder = SideInformationAutoEncoder(config.use_si_flag)


In [5]:
image_list = SideinformationImageImageList.from_csv(path="/mnt/code/repos/tDSIN/src/dsin/data",csv_names=["nano_KITTI_stereo_train.txt","nano_KITTI_stereo_val.txt"])

In [6]:
image_list.items[0]

'/mnt/code/repos/tDSIN/src/dsin/data/data_scene_flow_multiview/training/image_3/000000_10.png'

In [7]:
class CustomLoss(nn.Module):
    def forward(self, *args):
        raise Exception(args, args[1].shape)
    

In [8]:
ll = image_list.split_by_valid_func(lambda x: 'testing'  in x ).label_from_func(lambda x: x)

In [9]:
batchsize = 1
data = (image_list
        .split_by_valid_func(lambda x: 'testing'  in x )
        .label_from_func(lambda x: x)
        .transform(None,size=(336,1224),resize_method=ResizeMethod.CROP,tfm_y=True)
        .databunch(bs=batchsize))
 

In [10]:
data.train_ds[0][0]

ImageSiTuple [tensor([[[0.0237, 0.0235, 0.0235,  ..., 0.3647, 0.3647, 0.3647],
         [0.0286, 0.0237, 0.0237,  ..., 0.3647, 0.3651, 0.3649],
         [0.0510, 0.0275, 0.0275,  ..., 0.3649, 0.3723, 0.3684],
         ...,
         [0.0980, 0.0904, 0.0865,  ..., 0.0786, 0.0786, 0.0786],
         [0.0980, 0.0902, 0.0825,  ..., 0.0784, 0.0784, 0.0784],
         [0.0980, 0.0864, 0.0861,  ..., 0.0747, 0.0747, 0.0747]],

        [[0.0357, 0.0353, 0.0314,  ..., 0.4084, 0.4086, 0.4123],
         [0.0433, 0.0355, 0.0314,  ..., 0.4190, 0.4231, 0.4233],
         [0.0481, 0.0396, 0.0316,  ..., 0.4074, 0.4151, 0.4190],
         ...,
         [0.1096, 0.1098, 0.1098,  ..., 0.0941, 0.0943, 0.0943],
         [0.1098, 0.1061, 0.1061,  ..., 0.0829, 0.0829, 0.0866],
         [0.1098, 0.1059, 0.1059,  ..., 0.0861, 0.0824, 0.0825]],

        [[0.0308, 0.0310, 0.0273,  ..., 0.4654, 0.4656, 0.4624],
         [0.0196, 0.0235, 0.0235,  ..., 0.4392, 0.4433, 0.4551],
         [0.0210, 0.0247, 0.0237,  ..., 0.43

In [11]:
my_learner = Learner(data=data,
                     model=si_autoencoder,
                     opt_func=torch.optim.Adam,
                     loss_func=LossManager(config.use_si_flag))

In [ ]:


my_learner.fit(5)


epoch,train_loss,valid_loss,time
0,1065.512817,3072.430176,00:35
